## 3. Validation and Report Generation

The final phase of SDMT involves aggregating evidence, validating the metrics reflected by the evidence we collected, and displaying this information in a report.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

### CI Workaround

The CI will not have access to an OpenAI API key, so the `2f_evidence_time_behavior.ipynb` notebook must be skipped because it requires a key. If this notebook is not ran, this evidence is not filled causing this notebook to fail. So in the case that there is no API key, add a fake valid measurement here.

In [ ]:
import os

if os.environ.get("OPENAI_API_KEY") == None:
    from mlte.evidence.types.array import Array
    from mlte.measurement.external_measurement import ExternalMeasurement

    def fake_func():
        return []

    fake_measurement = ExternalMeasurement(
        "results returned promptly", Array, fake_func
    )
    fake_res = fake_measurement.evaluate()
    fake_res.save(force=True)

### Validate Evidence and get an updated `TestResults` with `Result`s

Now that we have our `TestSuite` ready and we have enough evidence, we create a `TestSuiteValidator` with our TestSuite, and add all the `Evidence`s we have. With that we can validate our tests and generate an output `TestResults`, with the validation results.

In [ ]:
from mlte.validation.test_suite_validator import TestSuiteValidator

# Load validator for default TestSuite id
test_suite_validator = TestSuiteValidator()

# Load all Evidence and validate TestCases
test_results = test_suite_validator.load_and_validate()

# We want to see the validation results in the Notebook, regardless of them being saved.
test_results.print_results()

# TestResults also supports persistence
test_results.save(force=True)

### Generate a Report

The final step of SDMT involves the generation of a report to communicate the results of model evaluation.

In [ ]:
from mlte.report.artifact import (
    Report,
    CommentDescriptor,
)

# Create a report with the default NegotiationCard, TestSuite and TestResults in this store.
report = Report(
    comments=[
        CommentDescriptor(
            content="This model should not be used for nefarious purposes."
        )
    ],
)

report.save(force=True, parents=True)